In [18]:
import pulp

In [19]:
# Crear un modelo de programación lineal entera
model = pulp.LpProblem("partidos_audiencia_max", pulp.LpMaximize)

In [20]:
# Creamos matriz de audiencias
audiencia = [ 
            [2, 1.3, 1],  
            [1.3, 0.9, 0.75], 
            [1, 0.75, 0.47], 
]

In [21]:
categorias = []

for i in range (20):

    if i < 3: # Categoría A
        categorias.append(0)
    elif i < 14: # Categoría B
        categorias.append(1)
    else: # Categoría C
        categorias.append(2)

print(categorias)

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]


In [22]:
penaliza_horario = [0.4,0.55,0.7,0.8,1,0.45,0.75,0.85,1,0.4]
# penaliza_horario = [1,1,1,1,1,1,1,1,1,1]

In [23]:
penaliza_coincidencia = {0:1, 1:1, 2:0.75,3:0.55,4:0.4,5:0.30,6:0.25,7:0.23,8:0.02,9:0.02}

In [24]:
# Definir las variables de decisión
# Tupla (local, visitante ,horario)
# 20 equipos
# 10 horarios

horario_partido_nofiltrada = [(local, visitante, horario) for local in range(20) for visitante in range(20) for horario in range(10)]

print(len(horario_partido_nofiltrada))

4000


In [25]:
# Definimos las variables de decisión
var_partido_horario = pulp.LpVariable.dicts("var_partido_horario", horario_partido_nofiltrada, 0, 1, cat=pulp.LpBinary)
var_numero_partidos_por_horario = pulp.LpVariable.dicts("var_numero_partidos_por_horario", [horario for horario in range (10)], lowBound=0, cat=pulp.LpInteger)


In [26]:
# Aplicamos la restricción de que un equipo no juega contra si mismo
for equipo in range(20):

    model += pulp.lpSum([var_partido_horario[equipo,equipo,horario]  for horario in range(10)]) == 0

In [27]:
# Aplicamos la restricción de que cada equipo solo juega un partido
for equipo in range(20):

    model += pulp.lpSum([var_partido_horario[equipo,visitante,horario] + var_partido_horario[visitante,equipo,horario] for visitante in range(20) for horario in range(10)]) == 1

In [28]:
# Aplicamos la restricción de que al menos un partido se programa en el horario de viernes y de lunes

#model += pulp.lpSum([var_partido_horario[local, visitante, 0] for local in range (20) for visitante in range (20)]) >=1
# model += pulp.lpSum([var_partido_horario[local, visitante, 9] for local in range (20) for visitante in range (20)]) >=1
model += var_numero_partidos_por_horario[0] >=1
model += var_numero_partidos_por_horario[9] >=1

In [29]:
# Restriccion de que la suma de partidos en horario tiene que ser igual al numero de partidos en ese horario
for horario in range(10):
    model += pulp.lpSum([var_partido_horario[local, visitante, horario] for local in range (20) for visitante in range (20)]) == var_numero_partidos_por_horario[horario]


In [30]:
# Restriccion de que la suma de partidos en horario tiene que ser igual al numero de partidos en ese horario
for horario in range(10):
    model += var_numero_partidos_por_horario[horario] <= 2 


In [31]:
# Creamos la función objetivo sumamos los valores de audiencia en función de la categoría del partido y aplica el coeficiente del horario


model += pulp.lpSum([var_partido_horario[(local, visitante, horario)] * audiencia[categorias[local]][categorias[visitante]]*penaliza_horario[horario] for local in range(20) for visitante in range(20) for horario in range(10)])



In [32]:
model.writeLP("partidos_audiencia_max.lp")

model.solve()

print("Estado:", pulp.LpStatus[model.status])
print("Valor óptimo:", pulp.value(model.objective))


Estado: Optimal
Valor óptimo: 8.206


In [33]:
def num_partidos_simultaneos(horario):
    # Número de partidos que comienzan en el horario
    partidos_simultaneos = int(pulp.lpSum([var_partido_horario[(local, visitante, horario)].value() for local in range(20) for visitante in range(20)]).value())
    return partidos_simultaneos

In [34]:
cantidad = [num_partidos_simultaneos(horario) for horario in range(10)]
print(cantidad)


[1, 0, 0, 2, 2, 0, 0, 2, 2, 1]


In [35]:
for local in range(20):
    for visitante in range(20):
        for horario in range(10):
            if var_partido_horario[local,visitante,horario].value() == 1:
                print(var_partido_horario[local,visitante,horario],"---->",var_partido_horario[local,visitante,horario].value())

for horario in range(10):
    
    print(var_numero_partidos_por_horario[horario],"---->",var_numero_partidos_por_horario[horario].value())

var_partido_horario_(0,_1,_4) ----> 1.0
var_partido_horario_(2,_7,_8) ----> 1.0
var_partido_horario_(3,_10,_8) ----> 1.0
var_partido_horario_(6,_15,_3) ----> 1.0
var_partido_horario_(11,_4,_4) ----> 1.0
var_partido_horario_(12,_5,_7) ----> 1.0
var_partido_horario_(13,_8,_7) ----> 1.0
var_partido_horario_(14,_17,_9) ----> 1.0
var_partido_horario_(18,_16,_0) ----> 1.0
var_partido_horario_(19,_9,_3) ----> 1.0
var_numero_partidos_por_horario_0 ----> 1.0
var_numero_partidos_por_horario_1 ----> 0.0
var_numero_partidos_por_horario_2 ----> 0.0
var_numero_partidos_por_horario_3 ----> 2.0
var_numero_partidos_por_horario_4 ----> 2.0
var_numero_partidos_por_horario_5 ----> 0.0
var_numero_partidos_por_horario_6 ----> 0.0
var_numero_partidos_por_horario_7 ----> 2.0
var_numero_partidos_por_horario_8 ----> 2.0
var_numero_partidos_por_horario_9 ----> 1.0
